In [ ]:
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport

## Load data

In [ ]:
# Load payroll data
# If these reports are run in one combined report with a payroll area identifier column there's no need to clean and concat them

m1_wage_df = pd.read_excel(wage_path_m1)  # list of all wage types with amounts paid for the full year, UNN
m2_wage_df = pd.read_excel(wage_path_m2) # list of all wage types with amounts paid for the full year, UNN  
m1_sick_pay_df = pd.read_excel(sick_path_m1) # list of all sickness payments made for the pay period, NUSL
m2_sick_pay_df = pd.read_excel(sick_path_m2) # list of all sickness payments made for the pay period, NUSL
absence_df = pd.read_excel(absence_path) # all absence types during the pay period  
leavers_df = pd.read_excel(leavers_path)  # all leavers during the pay period
stafflist_df = pd.read_excel(stafflist_path) # all contracted staff as of the census date
starters_df = pd.read_excel(starters_path) # starters as of the census date
gem_df = pd.read_excel(gem_path) # recipients of gem awards and long service

## Data Cleaning

### Create a mapping dictionary for recoding

In [ ]:
grade_mapping = {
    '01': '1',
    '02': '2',
    '03': '3',
    '04': '4',
    '05': '5',
    '06': '6',
    '07': '7',
    '08': '8',
    '09': '9',
    '10': '10',
    '11': '11',
    'CWD': 'MISC',
    'CWF': 'MISC',
    'CWB_N': 'MISC',
    'CWG': 'MISC',
    'APP': 'MISC',
    'CWB': 'MISC',
    'CWH': 'MISC',
    'CWE': 'MISC',
    'STP': 'MISC',
    'CWC': 'MISC',
    'DEM05': 'MISC',
    'VL': 'MISC',
    'INV': 'MISC',
    'EXTE': 'MISC',
    'VT': 'MISC',
    'AA05': 'MISC'
}

# Recode the 'Grade' column using the mapping
stafflist_df['Grade'] = stafflist_df['Grade'].map(grade_mapping)

In [ ]:
gem_df = gem_df.dropna(subset=['StaffNo'])
gem_df['StaffNo'].value_counts()

# Convert 'StaffNo' to integer
gem_df['StaffNo'] = gem_df['StaffNo'].astype(int)
# Drop rows where 'StaffNo' is NaN

In [ ]:
# Remove row 0 from stafflist
stafflist_df = stafflist_df.drop(0)
# Convert 'Personnel No' and 'Staff Id' columns to integers
stafflist_df['Personnel No'] = stafflist_df['Personnel No'].astype(int)
stafflist_df['Staff Id'] = stafflist_df['Staff Id'].astype(int)

In [ ]:
# Remove row 0 from absence
absence_df = absence_df.drop(0)
# Convert 'Assignment No' and 'Staff ID' columns to integers
absence_df['Assignment No'] = absence_df['Assignment No'].astype(int)
absence_df['Staff ID'] = absence_df['Staff ID'].astype(int)

### Create a mapping dictionary for sick pay

In [ ]:

column_mapping = dict(zip(m1_sick_pay_df.columns, m2_sick_pay_df.columns))

# Check if the mapping is as expected
print("\nColumn Mapping:")
for k, v in column_mapping.items():
    print(f"{k} will be renamed to {v}")

# Rename columns in m1_wage_df using the mapping
m1_sick_pay_df.rename(columns=column_mapping, inplace=True)

# Add 'Payroll_Area' column with value 'M1' to m1_wage_df
m1_sick_pay_df['Payroll_Area'] = 'M1'

# Add 'Payroll_Area' column with value 'M2' to m2_wage_df
m1_sick_pay_df['Payroll_Area'] = 'M2'

# Concat M1 and M2
sick_pay_df = pd.concat([m1_sick_pay_df, m2_sick_pay_df], ignore_index=True)

### Create a mapping dictionary for wage

In [ ]:
column_mapping = dict(zip(m1_wage_df.columns, m2_wage_df.columns))

# Check if the mapping is as expected
print("\nColumn Mapping:")
for k, v in column_mapping.items():
    print(f"{k} will be renamed to {v}")

# Rename columns in m1_wage_df using the mapping
m1_wage_df.rename(columns=column_mapping, inplace=True)

# Add 'Payroll_Area' column with value 'M1' to m1_wage_df
m1_wage_df['Payroll_Area'] = 'M1'

# Add 'Payroll_Area' column with value 'M2' to m2_wage_df
m2_wage_df['Payroll_Area'] = 'M2'

# Concat M1 and M2
mergewage_df = pd.concat([m1_wage_df, m2_wage_df], ignore_index=True)

# Drop total gross
mergewage_df = mergewage_df[mergewage_df['Wage_Text'] != 'Total gross']

### add GEM awards to bonuses

In [ ]:
# Transform 'StaffNo' in gem_df for 'Personnel_No'
gem_df['Personnel_No'] = gem_df['StaffNo'].astype(str) + '000'
gem_df['Personnel_No'] = gem_df['Personnel_No'].astype(int)
# Rename columns in gem_df to match wage_df
gem_df = gem_df.rename(columns={'StaffNo': 'Staff_Id',
                                'Wage_Text': 'Wage_Text',
                                'Amount': 'Amount',
                                'Date': 'Payment_Date'})

# Selecting only the relevant columns
relevant_columns = ['Staff_Id', 'Personnel_No', 'Wage_Text', 'Amount', 'Payment_Date']
gem_df_transformed = gem_df[relevant_columns]

In [ ]:
mergewage_df['Wage_Text'].unique()

### Prepare for merging staff lists and payroll data

In [ ]:
stafflist_df.columns = stafflist_df.columns.str.replace(" ", "_").str.replace("/", "_")
overlap_columns = stafflist_df.columns.intersection(mergewage_df.columns).tolist()
stafflist_df = stafflist_df.rename(columns={col: col + "_SL" for col in overlap_columns if col != 'Personnel_No'})

In [ ]:
newmergewage_df = pd.concat([mergewage_df, gem_df_transformed], ignore_index=True)

In [ ]:
newmergewage_df['Wage_Text'].unique()

<div class="alert alert-block alert-info">
note we merge to the stafflist and not the other way round, so we can eliminate casual payments in wage_df 
to workers who aren't contracted (i.e. are not present in stafflist_df)</br></div>

In [ ]:
newmergewage_df['Personnel_No'] = newmergewage_df['Personnel_No'].astype(int)
wage_df = pd.merge(stafflist_df, newmergewage_df, how='left', on=['Personnel_No'])

<div class="alert alert-block alert-info">
Drop any rows in wage_df that don't have a Payroll_Area_SL from stafflist_df, indicating not employees (regular or casual)</div>

In [ ]:
wage_df = wage_df.dropna(subset=['Payroll_Area_SL'])

In [ ]:
wage_amounts = wage_df.groupby('Wage_Text')['Amount'].describe()

### Set payment year start and end dates

In [ ]:

start_date = pd.Timestamp('2022-04-01')
end_date = pd.Timestamp('2023-03-31')

# Filter the dataframe based on the date range
wage_df = wage_df[
    (wage_df['Payment_Date'] >= start_date) & 
    (wage_df['Payment_Date'] <= end_date)
]

__[gov.uk preparing your data](https://www.gov.uk/government/publications/gender-pay-gap-reporting-guidance-for-employers/preparing-your-data)__

<div class="alert alert-block alert-info">For salaried staff we can use FTE</br></br>
For employees who are not contracted to work the same number of hours each week, 
you need to work out a mean (average) for them.
use the 12 weeks that ends with the last full week of the pay period that includes your snapshot date
take the total number of hours worked by each employee over this period and divide it by 12
If the 12-week period includes a week where the employee did not work at all, 
replace it with an earlier week where they worked.</div>

## Separate Relevant and Full-pay Relevant

<div class="alert alert-block alert-info">Sickness exclusions</br></br>
Note we can't filter Absence Year on 2022/23 in case of long term absences that may have started in the previous year</div>

In [ ]:
# Convert 'Date' in sick_pay_df to datetime
sick_pay_df['Date'] = pd.to_datetime(sick_pay_df['Date'])

# Filter for the specific date range
start_date = '2023-03-01'
end_date = '2023-03-31'
sick_pay_df_filtered = sick_pay_df[(sick_pay_df['Date'] >= start_date) & (sick_pay_df['Date'] <= end_date)]

# Identify individuals who received sick pay in the specified period
sick_individuals = sick_pay_df_filtered['Personnel_No'].unique()

# Flag these individuals in the wage data
wage_df['Sickness_Exclusion'] = wage_df['Personnel_No'].isin(sick_individuals)

<div class="alert alert-block alert-info">Absence exclusions. Strike days don't count even though unpaid</div>

In [ ]:
# Convert 'Start Date' to datetime, coerce out-of-bounds dates to NaT
absence_df['Start Date'] = pd.to_datetime(absence_df['Start Date'], errors='coerce')

# Convert 'End Date' to datetime, coerce out-of-bounds dates to NaT
absence_df['End Date'] = pd.to_datetime(absence_df['End Date'], errors='coerce')

# Define the list of absence types to filter
absence_types = [
    'Career Break', 'Special Leave', 'Maternity leave 2022',
    'Unpaid Leave', 'Shared Parental (Paid)', 'Shared Parental (Unpaid)', 'Paternity leave 2022'
]
# Filter absence_df and create absence_reduced_pay. Picks up anyone with reduced pay due to absence during the pay period
absence_reduced_pay = absence_df[
    (absence_df['Absence Type'].isin(absence_types)) &
    (absence_df['Start Date'] <= pd.Timestamp('2023-03-31')) &
    ((absence_df['End Date'] >= pd.Timestamp('2023-03-01')) | absence_df['End Date'].isna())
]

# Step 4: Create 'Absence_Exclusion' in wage_df
# True if 'Personnel No' in wage_df is in 'Assignment No' in absence_reduced_pay
wage_df['Absence_Exclusion'] = wage_df['Personnel_No'].isin(absence_reduced_pay['Assignment No'])

<div class="alert alert-block alert-info">Starter exclusions - won't have been paid full salary for pay period</div>

In [ ]:
starters_df['Latest Hire Date'] = pd.to_datetime(starters_df['Latest Hire Date'])

# Define start and end date for the filter
start_date = pd.Timestamp('2023-03-01')
end_date = pd.Timestamp('2023-03-31')

# Step 2: Filter starters_df based on the date range
filtered_starters_df = starters_df[
    (starters_df['Latest Hire Date'] >= start_date) & 
    (starters_df['Latest Hire Date'] <= end_date)
]

# Step 3: Create 'Starter_Exclusion' in wage_df
# True if 'Personnel No' in wage_df is in 'Personnel_No' in filtered_starters_df
wage_df['Starter_Exclusion'] = wage_df['Personnel_No'].isin(filtered_starters_df['Personnel No'])

In [ ]:
# Check exclusion counts
sickness_exclusion_counts = wage_df['Sickness_Exclusion'].value_counts()
absence_exclusion_counts = wage_df['Absence_Exclusion'].value_counts()
starter_exclusion_counts = wage_df['Starter_Exclusion'].value_counts()

value_counts_table = pd.DataFrame({
    'Sickness_Exclusion': sickness_exclusion_counts,
    'Absence_Exclusion': absence_exclusion_counts,
    'Starter_Exclusion': starter_exclusion_counts
})

value_counts_table = value_counts_table.reset_index()
value_counts_table.columns = ['Value', 'Sickness_Exclusion', 'Absence_Exclusion', 'Starter_Exclusion']

<div class="alert alert-block alert-info">Classify as relevant or full-pay relevant</div>

In [ ]:
import numpy as np

wage_df['Employee_Classification'] = np.where(
    (wage_df['Sickness_Exclusion'] | wage_df['Absence_Exclusion'] | wage_df['Starter_Exclusion']),
    'Relevant',
    'Full-pay relevant'
)

In [ ]:
wage_df['Employee_Classification'].value_counts() # includes duplicate personnel nos as no filter yet on dates

## Classify wage type codes

In [ ]:
salary_sacrifice_types = [
    'Cycle Scheme SS', 'Nursery SS', 'Holiday SS', 
    'Employee Pen Cont US5'
]

ordinary_pay_types = [
    'Monthly Rotate.Shift All.', 'Monthly Altern.Shift All.', 'Salary', 
    'Protected Pay', 'Honorarium', 'Standby',
    'Additional Payments', 'Childcare Voucher' # childcare voucher goes in here and not SS
]

ordinary_casual_types = [
    'Casual Hrs -CWA-CWH', 'Casual Hours', 'Invigilator Supervisor', 'Invigilator',
    'Variable Hours', 'Hourly @ 1', 'Academic Hourly',
    'AL Marking'
]

bonus_pay_types = [
    'Retention Supplement', 'Performance Pay', 'Bonus', 'Recruitment Supplement', 'GEM'
]

wage_df['Wage_Class'] = wage_df['Wage_Text'].apply(
    lambda x: 'Bonus_Pay' if x in bonus_pay_types else (
        'Ordinary_Casual' if x in ordinary_casual_types else (
            'Ordinary_Pay' if x in ordinary_pay_types else (
                'Salary_Sacrifice' if x in salary_sacrifice_types else 'Other'
                )
            )
        )
    )

### Check unique values

In [ ]:
unique_wage_text = set(wage_df['Wage_Text'].unique())

# Union of sets of values in the lists
all_wage_types = set(salary_sacrifice_types + ordinary_pay_types + ordinary_casual_types + bonus_pay_types)

# Check if all unique values are represented in only one list
if unique_wage_text == all_wage_types:
    print("All unique values in wage_df['Wage_Text'] are represented in one and only one list.")
else:
    print("Not all unique values in wage_df['Wage_Text'] are represented in one and only one list.")

In [ ]:
# Check that there are hours recorded in all ordinary_casual_types as we don't have FTE
bonus_pay_df = wage_df[wage_df['Wage_Text'].isin(bonus_pay_types)]
# Apply describe() to 'Number' column grouped by 'Wage_Text'
describe_stats = bonus_pay_df.groupby('Wage_Text')['Amount'].describe()

<div class="alert alert-block alert-info"> Check FTEs for ordinary pay</div>

In [ ]:
ordinary_pay_df = wage_df[wage_df['Wage_Text'].isin(ordinary_pay_types)]
describe_stats = ordinary_pay_df.groupby('Wage_Text')['FTE'].describe()

<div class="alert alert-block alert-info">Check that there are hours recorded in all ordinary_casual_types as we don't have FTE</div>

In [ ]:
ordinary_casual_df = wage_df[wage_df['Wage_Text'].isin(ordinary_casual_types)]
# Apply describe() to 'Number' column grouped by 'Wage_Text'
describe_stats = ordinary_casual_df.groupby('Wage_Text')['Number'].describe()

## Reclassify bonus pay that should be ordinary pay

<div class="alert alert-block alert-info">for any bonus_pay_types that appear in the last three months for the same NINO, reclassify these as ordinary pay and not bonus
</br>
https://www.gov.uk/government/publications/gender-pay-gap-reporting-guidance-for-employers/preparing-your-data#step-2-add-ordinary-pay</br>

You must include all allowances in ordinary pay. In some cases, you will need to decide whether a payment is an allowance or an expense.</br>

You should include:</br>

any allowances your employees would expect to receive in their regular pay</br>
recruiting and retaining an employee – but treat ‘one off’ incentive payments as bonus pay, 
not an allowance, if you pay them at the start of employment or they are closer to a bonus than an ongoing allowance</div>

In [ ]:
# Create a new column 'Month_Year' to represent the month and year of 'Payment_Date'
wage_df['Month_Year'] = wage_df['Payment_Date'].dt.to_period('M')

# Create a mask for bonus payments in the specified period
bonus_mask = (wage_df['Wage_Text'].isin(bonus_pay_types)) & (wage_df['Payment_Date'] >= '2022-04-01') & (wage_df['Payment_Date'] <= '2023-03-31')

# Group by 'Personnel_No' and 'Wage_Text' and count unique 'Month_Year' values
counts = wage_df[bonus_mask].groupby(['Personnel_No', 'Wage_Text'])['Month_Year'].nunique().reset_index()

# Filter for 'Personnel_No' with three or more months of the same 'Wage_Text'
qualified_personnel = counts[counts['Month_Year'] >= 3]

# Update 'Wage_Class' for qualified personnel and 'bonus_pay_types'
wage_df.loc[
    (wage_df['Personnel_No'].isin(qualified_personnel['Personnel_No'])) &
    (wage_df['Wage_Text'].isin(bonus_pay_types)),
    'Wage_Class'
] = 'Ordinary_Pay'

# Remove the 'Month_Year' column if it's no longer needed
wage_df.drop(columns=['Month_Year'], inplace=True)

changed_wage_class = wage_df[(wage_df['Wage_Class'] == 'Ordinary_Pay') &
                            (wage_df['Wage_Text'].isin(bonus_pay_types))]

# Get unique 'Personnel_No' values
changed_personnel_nos = changed_wage_class['Personnel_No'].unique()

In [ ]:
wage_df['Wage_Text'].unique()

### Count unique payment dates for each Personnel_No for bonus payments

In [ ]:
bonus_payment_counts = wage_df[wage_df['Wage_Class'] == 'Bonus_Pay'].groupby('Personnel_No')['Payment_Date'].nunique()

# Identify Personnel_No with more than one bonus payment date
monthly_bonus_personnel = bonus_payment_counts[bonus_payment_counts > 1].index

# Set 'Bonus_Period_Months' based on the count
# Default to annual (12 months)
wage_df['Bonus_Period_Months'] = 12

# Set to monthly (1 month) for those identified with more than one payment date
wage_df.loc[wage_df['Personnel_No'].isin(monthly_bonus_personnel) & (wage_df['Wage_Class'] == 'Bonus_Pay'), 'Bonus_Period_Months'] = 1

# GEM period is 1 month as they can be awarded in multiple months
wage_df['Bonus_Period_Months'] = np.where(wage_df['Wage_Text'] == 'GEM', 1, wage_df['Bonus_Period_Months'])

# Constants
DAYS_IN_MONTH = 30.44
PAY_DATE = pd.Timestamp('2023-03-24')
START_DATE = pd.Timestamp('2023-03-01')
END_DATE = pd.Timestamp('2023-03-31')

# Identify Bonus Payments in the Pay Period
pay_period_bonus = wage_df[
    (wage_df['Wage_Class'] == 'Bonus_Pay') &
    (wage_df['Payment_Date'] >= START_DATE) &
    (wage_df['Payment_Date'] <= END_DATE)
]

# Filter for relevant bonus payments around the snapshot date
relevant_bonuses = wage_df[
    (wage_df['Wage_Class'] == 'Bonus_Pay') &
    (wage_df['Payment_Date'] == PAY_DATE) & 
    (~wage_df['Starter_Exclusion']) &  # Full-pay relevant employees (not excluded)
    (~wage_df['Sickness_Exclusion']) &
    (~wage_df['Absence_Exclusion'])
]

# After setting 'Bonus_Period_Months' as previously described, calculate the prorated bonus
pay_period_bonus['Prorated_Bonus_Total'] = (
    (pay_period_bonus['Amount'] / (pay_period_bonus['Bonus_Period_Months'] * DAYS_IN_MONTH)) * DAYS_IN_MONTH
)

In [ ]:
wage_df = wage_df.merge(pay_period_bonus[['Personnel_No', 'Prorated_Bonus_Total']], on='Personnel_No', how='left')
wage_df['Wage_Class'].value_counts()

In [ ]:
wage_df = wage_df[wage_df['Wage_Class'] != 'Other']

In [ ]:
wage_df['Wage_Text'].value_counts()

In [ ]:
# Get combinations of 'Wage_Class' and 'Wage_Text' and count occurrences
combination_counts = wage_df.groupby(['Wage_Class', 'Wage_Text', 'Payment_Date']).size().reset_index(name='Count')

##  Create table of bonuses with relevant and full-pay relevant

In [ ]:
# Pivot for Total_Bonus_Pay and Total_Bonus_Count
bonus_pay_df = wage_df[(wage_df['Wage_Class'] == 'Bonus_Pay') & 
                       (wage_df['Payment_Date'] >= '2022-04-01') & 
                       (wage_df['Payment_Date'] <= '2023-03-31')]
pivot_bonus_pay = bonus_pay_df.pivot_table(index='Personnel_No', 
                                           values='Amount', 
                                           aggfunc={'Amount': 'sum', 'Personnel_No': 'count'}, 
                                           fill_value=0)
pivot_bonus_pay.rename(columns={'Amount': 'Full_Total_Bonus', 'Personnel_No': 'Total_Bonus_Count'}, inplace=True)

bonus_df = pd.merge(pivot_bonus_pay, stafflist_df, on='Personnel_No', how='left')
bonus_df = bonus_df.sort_values(by='Full_Total_Bonus', ascending=False)
bonus_df_unique_personnel_no = bonus_df.drop_duplicates(subset=['Personnel_No'], keep='first')
bonus_df = bonus_df_unique_personnel_no.fillna(0)
bonus_df['Wage_Class'] = 'Bonus_Pay'

In [ ]:
got_bonus_df = bonus_df[bonus_df['Full_Total_Bonus'] > 0]
got_bonus_df['Full_Total_Bonus'].describe()

In [ ]:
# Count the number of unique Personnel_No
unique_personnel_no_count = bonus_df['Personnel_No'].nunique()

# Filter for personnel with Full_Total_Bonus > 0
bonus_df_filtered = bonus_df[bonus_df['Full_Total_Bonus'] > 0]

# Count and calculate percentage of personnel with bonus > 0
personnel_with_bonus_count = bonus_df_filtered['Personnel_No'].nunique()
personnel_with_bonus_percentage = (personnel_with_bonus_count / unique_personnel_no_count) * 100

# Print results
print(f"Number of unique Personnel_No: {unique_personnel_no_count}")
print(f"Count of personnel with Full_Total_Bonus > 0: {personnel_with_bonus_count}")
print(f"Percentage of personnel with Full_Total_Bonus > 0: {personnel_with_bonus_percentage:.2f} %")

##  Save the data to Excel before it's filtered to the payment period

In [ ]:
from openpyxl import Workbook
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl.utils.dataframe import dataframe_to_rows

def create_excel_with_tables(dfs, sheet_names, file_path):
    """
    Create an Excel file with multiple sheets, each containing a DataFrame formatted as a table.

    :param dfs: List of DataFrames.
    :param sheet_names: List of sheet names corresponding to the DataFrames.
    :param file_path: Path to save the Excel file.
    """
    if len(dfs) != len(sheet_names):
        raise ValueError("The number of DataFrames and sheet names must be the same")

    # Create a new workbook
    wb = Workbook()

    for df, sheet_name in zip(dfs, sheet_names):
        # Create a new sheet
        ws = wb.create_sheet(title=sheet_name)

        # Adding DataFrame to sheet
        for r_idx, row in enumerate(dataframe_to_rows(df, index=False, header=True), 1):
            for c_idx, value in enumerate(row, 1):
                ws.cell(row=r_idx, column=c_idx, value=value)

        # Create a table
        table = Table(displayName=sheet_name, ref=ws.dimensions)

        # Add a predefined style
        style = TableStyleInfo(name="TableStyleLight8", showFirstColumn=False,
                               showLastColumn=False, showRowStripes=True, showColumnStripes=True)
        table.tableStyleInfo = style
        ws.add_table(table)

    # Remove the default sheet created by openpyxl
    if 'Sheet' in wb.sheetnames:
        wb.remove(wb['Sheet'])

    # Save the workbook
    wb.save(file_path)

## Now the bonuses have been prorated we need to filter by date range

In [ ]:
filtered_wage_df = wage_df[(wage_df['Payment_Date'] >= START_DATE) & (wage_df['Payment_Date'] <= END_DATE)]

# Extract the unique Personnel_No values from stafflist_df
valid_personnel_nos = stafflist_df['Personnel_No'].unique()

# Drop anyone who isn't an employee
filtered_wage_df = filtered_wage_df[filtered_wage_df['Personnel_No'].isin(valid_personnel_nos)]

In [ ]:
wage_df['Wage_Class'].unique()

In [ ]:
from datetime import datetime, timedelta

# Pivot the DataFrame and sum both 'Amount' and 'Number' columns
pivot_df = filtered_wage_df.pivot_table(index='Personnel_No', 
                               columns='Wage_Class', 
                               values=['Amount', 'Number'], 
                               aggfunc={'Amount': 'sum', 'Number': 'sum'}, 
                               fill_value=0)

# Reset the index to make 'Personnel_No' a column
pivot_df.reset_index(inplace=True)

pivot_df.columns = ['_'.join(col).strip() for col in pivot_df.columns.values]

# Rename 'Personnel_No_'
pivot_df.rename(columns={'Personnel_No_': 'Personnel_No'}, inplace=True)

# Change 'Amount_' to 'Total_'
pivot_df.rename(columns={col: col.replace('Amount_', 'Total_') for col in pivot_df.columns if col.startswith('Amount_')}, inplace=True)

# Rename 'Number_Ordinary_Casual'
pivot_df.rename(columns={'Number_Ordinary_Casual': 'Total_Casual_Hours'}, inplace=True)

# Drop unwanted columns
pivot_df.drop(columns=['Number_Bonus_Pay', 'Number_Ordinary_Pay', 'Number_Salary_Sacrifice'], inplace=True)

# Define the date range for the 12-week period prior to March 31, 2023
end_date = pd.to_datetime('2023-03-31')
start_date = end_date - pd.Timedelta(weeks=12)

# Filter wage_df for the 12-week period
filtered_wage_df_12weeks = wage_df[(wage_df['Payment_Date'] >= start_date) & (wage_df['Payment_Date'] <= end_date)]

# Identify Personnel_No with 'Ordinary_Casual' payments (where Wage_Class is "Ordinary_Casual" and the amount is > 0) and 'Number' > 0 in March 2023
march_personnel = wage_df[(wage_df['Payment_Date'] >= '2023-03-01') & 
                          (wage_df['Payment_Date'] <= '2023-03-31') & 
                          (wage_df['Wage_Class'] == 'Ordinary_Casual') & 
                          (wage_df['Amount'] > 0) & 
                          (wage_df['Number'] > 0)]['Personnel_No'].unique()

# Filter the 12-week DataFrame for these Personnel_No
filtered_wage_df_12weeks = filtered_wage_df_12weeks[filtered_wage_df_12weeks['Personnel_No'].isin(march_personnel)]

# Sum and average Total_Ordinary_Casual and Total_Casual_Hours over the 12 weeks
average_wage = filtered_wage_df_12weeks[filtered_wage_df_12weeks['Wage_Class'] == 'Ordinary_Casual'].groupby('Personnel_No').agg({
    'Amount': 'sum',
    'Number': 'sum' 
}).reset_index()
average_wage['Average_Casual_Pay'] = average_wage['Amount'] / 3  # Dividing by 3 to average over 12 weeks
average_wage['Average_Casual_Hours'] = average_wage['Number'] / 3

# Merge with wage_df_merged to overwrite Total_Casual_Pay and Total_Casual_Hours
wage_df_merged = filtered_wage_df.merge(average_wage[['Personnel_No', 'Average_Casual_Pay', 'Average_Casual_Hours']], on='Personnel_No', how='left')

# Replace values in wage_df_merged
wage_df_merged.loc[wage_df_merged['Personnel_No'].isin(march_personnel), 'Total_Ordinary_Casual'] = wage_df_merged['Average_Casual_Pay']
wage_df_merged.loc[wage_df_merged['Personnel_No'].isin(march_personnel), 'Total_Casual_Hours'] = wage_df_merged['Average_Casual_Hours']

# Drop the temporary average columns
wage_df_merged.drop(columns=['Average_Casual_Pay', 'Average_Casual_Hours'], inplace=True)


In [ ]:
ordinary_casual_pay = wage_df_merged['Total_Ordinary_Casual'].describe()

In [ ]:
ordinary_casual_hours = wage_df_merged['Total_Casual_Hours'].describe()

In [ ]:
wagepivot_df = filtered_wage_df.merge(pivot_df, on='Personnel_No', how='left')
wage_nodupe_df = wagepivot_df.drop_duplicates(subset='NINO')
wage_nodupe_df.drop_duplicates(subset='Personnel_No', inplace=True)

In [ ]:
## BONUS CALCS HERE
wage_nodupe_df['Prorated_Bonus_Total'].describe()


# Pivot for Total_Bonus_Pay and Total_Bonus_Count
bonus_pay_df = wage_nodupe_df[(wage_nodupe_df['Wage_Class'] == 'Bonus_Pay') & 
                       (wage_nodupe_df['Payment_Date'] >= '2022-04-01') & 
                       (wage_nodupe_df['Payment_Date'] <= '2023-03-31')]
pivot_bonus_pay = bonus_pay_df.pivot_table(index='Personnel_No', 
                                           values='Amount', 
                                           aggfunc={'Amount': 'sum', 'Personnel_No': 'count'}, 
                                           fill_value=0)
pivot_bonus_pay.rename(columns={'Amount': 'Full_Total_Bonus', 'Personnel_No': 'Total_Bonus_Count'}, inplace=True)

# Merge pivoted data back into wage_df
wage_nodupe_df = wage_nodupe_df.merge(pivot_bonus_pay, on='Personnel_No', how='left')

<div class="alert alert-block alert-info">Check against National Minimum Wage or National Living Wage. 
Uncomment the last line below to apply a NMW floor to hourly rate</div>

In [ ]:
from datetime import datetime
# Calculate NMW per individual
wage_nodupe_df['Date_of_Birth'] = pd.to_datetime(wage_nodupe_df['Date_of_Birth'])

# Define NMW rates for March 2023
nmw_rates = {
    '23 and over': 9.50,
    '21 to 22': 9.18,
    '18 to 20': 6.83
}

# Reference date: March 31, 2023
reference_date = datetime(2023, 3, 31)

# Calculate age on reference date
wage_nodupe_df['Age_on_Reference'] = wage_nodupe_df['Date_of_Birth'].apply(lambda dob: reference_date.year - dob.year - ((reference_date.month, reference_date.day) < (dob.month, dob.day)))

# Assign NMW based on age
wage_nodupe_df['NMW'] = wage_nodupe_df['Age_on_Reference'].apply(
    lambda age: nmw_rates['23 and over'] if age >= 23 else (
        nmw_rates['21 to 22'] if age >= 21 else (
            nmw_rates['18 to 20'] if age >= 18 else 0
        )
    )
)

#wage_nodupe_df[['Age_on_Reference', 'NMW']]

#wage_nodupe_df['HourlyExcSS'] = wage_nodupe_df['HourlyExcSS'].apply(lambda x: max(x, NMW))

##  Calculate Hourly rate inclusive and exclusive of salary sacrifice

In [ ]:
# Ensure that FTE is a numeric and not a string or object type
wage_nodupe_df['FTE'] = pd.to_numeric(wage_nodupe_df['FTE'], errors='coerce')

# Calculate WeeklyHours as a percentage of 37 based on FTE for ordinary staff
wage_nodupe_df.loc[wage_nodupe_df['FTE'] > 0, 'WeeklyHours'] = (37 * wage_nodupe_df['FTE']) / 100

# Convert 'Total_Ordinary_Casual' and 'Number' to numeric, handling non-numeric values
wage_nodupe_df['Total_Ordinary_Casual'] = pd.to_numeric(wage_nodupe_df['Total_Ordinary_Casual'], errors='coerce')
wage_nodupe_df['Number'] = pd.to_numeric(wage_nodupe_df['Number'], errors='coerce')

# Calculate HourlyExcSS based on the type of staff (ordinary or casual)
# For ordinary staff (FTE > 0), use the existing formula
# For casual staff (FTE == 0 or NaN), use Total_Ordinary_Casual / Total_Casual_Hours
wage_nodupe_df['HourlyExcSS'] = np.where(
    wage_nodupe_df['FTE'] > 0,
    (wage_nodupe_df['Total_Ordinary_Pay'] * 7) / (30.44 * wage_nodupe_df['WeeklyHours']),
    wage_nodupe_df['Total_Ordinary_Casual'] / wage_nodupe_df['Total_Casual_Hours']
)

# Handle division by zero or NaN values if 'Number' column can have 0 or NaN values
wage_nodupe_df['HourlyExcSS'] = wage_nodupe_df['HourlyExcSS'].replace([np.inf, -np.inf], np.nan)

# Calculate HourlyIncSS based on the type of staff (ordinary or casual)
wage_nodupe_df['HourlyIncSS'] = np.where(
    wage_nodupe_df['FTE'] > 0,
    ((wage_nodupe_df['Total_Ordinary_Pay'] + wage_nodupe_df['Total_Salary_Sacrifice']) * 7) / (30.44 * wage_nodupe_df['WeeklyHours']),
    (wage_nodupe_df['Total_Ordinary_Casual'] + wage_nodupe_df['Total_Salary_Sacrifice']) / wage_nodupe_df['Total_Casual_Hours']
) # add the total as SS deducted from gross pay is negative

# Handle division by zero or NaN values if 'HourlyIncSS' column can have 0 or NaN values
wage_nodupe_df['HourlyIncSS'] = wage_nodupe_df['HourlyIncSS'].replace([np.inf, -np.inf], np.nan)

In [ ]:
wage_nodupe_df['Employee_Classification'].unique()

<div class="alert alert-block alert-info">Do check on minimum hourly rates and 
any individuals who are not full-pay relevant, using inc salary sacrifice</div>

In [ ]:
wage_nodupe_df['NMW_Check'] = wage_nodupe_df['HourlyIncSS'] < wage_nodupe_df['NMW']
nmw_true_rows = wage_nodupe_df[wage_nodupe_df['NMW_Check'] == True]
# Set 'Employee_Classification' to 'Relevant' where 'NMW_Check' is True
wage_nodupe_df.loc[wage_nodupe_df['NMW_Check'] == True, 'Employee_Classification'] = 'Relevant'

## Add domicile, ethnicitye2 and ethnicitye5 columns

<div class="alert alert-block alert-info">create UKNonUK (domicile)</div>

In [ ]:
wage_nodupe_df['Nationality'].value_counts()

def map_domicile(column):
    # Lowercase the input for case-insensitive comparison
    lowercased = column.str.lower()

    # Define the mapping conditions
    conditions = [
        lowercased.str.contains('refused|not provided|prefer not to say|not available|unknown'),
        lowercased.str.contains('british'),
    ]

    # Define the mapping values
    values = ['Unknown','UK']

    # Apply the mapping
    return pd.Series(np.select(conditions, values, default='NonUK'), index=column.index)

wage_nodupe_df['UKNonUK'] = map_domicile(wage_nodupe_df['Nationality'])

In [ ]:
wage_nodupe_df['UKNonUK'].value_counts()

<div class="alert alert-block alert-info">create 2 factor ethnicity</div>

In [ ]:
def map_ethnicitye2(column):
    # Lowercase the input for case-insensitive comparison
    lowercased = column.str.lower()

    # Define the mapping conditions
    conditions = [
        lowercased.str.contains('refused|not provided|prefer not to say|not available|unknown'),
        lowercased.str.contains('asian|chinese') & ~lowercased.str.contains('mixed'),
        lowercased.str.contains('black') & ~lowercased.str.contains('mixed'),
        lowercased.str.contains('mixed'),
        lowercased.str.contains('white') & ~lowercased.str.contains('mixed|gypsy|traveller'),
        lowercased.str.contains('gypsy|traveller'),
        lowercased.str.contains('arab'),
        lowercased.str.contains('other') & ~lowercased.str.contains('white|black|mixed|asian')
    ]

    # Define the mapping values
    values = ['Unknown', 'BAME', 'BAME', 'BAME', 'White', 'White', 'BAME', 'BAME']

    # Apply the mapping
    return pd.Series(np.select(conditions, values, default='Other'), index=column.index)

wage_nodupe_df['EthnicityE2'] = map_ethnicitye2(wage_nodupe_df['Ethnicity'])

In [ ]:
wage_nodupe_df['EthnicityE2'].value_counts()

In [ ]:
ethnicity_values = ['Unknown']
ethnicity_df = wage_nodupe_df.loc[wage_nodupe_df['EthnicityE2'].isin(ethnicity_values), ['Ethnicity', 'EthnicityE2']]

<div class="alert alert-block alert-info">create 5 factor ethnicity</div>

In [ ]:
def map_ethnicitye5(column):
    # Lowercase the input for case-insensitive comparison
    lowercased = column.str.lower()

    # Define the mapping conditions
    conditions = [
        lowercased.str.contains('refused|not provided|prefer not to say|not available|unknown'),
        lowercased.str.contains('asian|chinese') & ~lowercased.str.contains('mixed'),
        lowercased.str.contains('black') & ~lowercased.str.contains('mixed'),
        lowercased.str.contains('mixed'),
        lowercased.str.contains('white') & ~lowercased.str.contains('mixed|gypsy|traveller'),
        lowercased.str.contains('gypsy|traveller'),
        lowercased.str.contains('arab'),
        lowercased.str.contains('other') & ~lowercased.str.contains('white|black|mixed|asian')
    ]

    # Define the mapping values
    values = ['Unknown', 'Asian', 'Black', 'Mixed', 'White', 'White', 'Other', 'Other']

    # Apply the mapping
    return pd.Series(np.select(conditions, values, default='Other'), index=column.index)

wage_nodupe_df['EthnicityE5'] = map_ethnicitye5(wage_nodupe_df['Ethnicity'])
# Define the custom order
custom_order = ['Asian', 'Black', 'Mixed', 'Other', 'White', 'Unknown']

# Convert 'EthnicityE5' to a categorical column with custom order
wage_nodupe_df['EthnicityE5'] = pd.Categorical(wage_nodupe_df['EthnicityE5'], categories=custom_order, ordered=True)

## Save the data to Excel now its been filtered to the payment period

In [ ]:
import os
wage_nodupe_relandfprel_df = wage_nodupe_df
# Full-pay relevant only from here
wage_nodupe_df = wage_nodupe_df[wage_nodupe_df['Employee_Classification'] != 'Relevant']

columns_to_drop = [
    'Title', 'Forename', 'Preferred_Forename', 'Surname', 'Contract', 
    'Date_First_Hired', 'Latest_Hire_Date', 
    'Projected_End_Date', 'Current_Assignment', 
    'Position', 'Position_ID', 'Job', 'Responsibility', 'Responsibility_Start_Date', 
    'Responsibility_End_Date', 'Line_Manager_(Current)', 'Organisation_Level_0', 
    'Organisation_Section', 'Doctorally_Qualified', 'HEA', 'HEA_Category', 'CRB_Status', 
    'Working_Time', 'Working_Hours_per_Week', 'Grade_Point', 
    'Annual_Base_Salary', 'FTE_Annual_Base_Salary', 'Contracted_Salary', 'NI_Number', 'Pension_Scheme', 'Master_CC', 
    'Dept_-_Master_CC', 'HESA_ACC', 'REF_Eligible', 'REF\'able', 'Site', 
    'Onsite_Location', 'User_ID', 'Email', 'DLA_Summary', 'DLA_Category', 'SOC_Code', 
    'Company', 'Status', 'Organisation_Sub_Group', 'Group_5_Desc', 'Surname_Name', 
    'Name', 'NINO', 'Comp_Code', 'Comp_Name', 'Pers_Area', 'Pers_Area_Text', 
    'Payroll_Area', 'Payroll_Text', 'Period_Params', 'Period_Params_Name', 
    'For_Period', 'Payment_Date', 'Grouping', 'Wage_Type', 'Wage_Class', 'Wage_Text', 
    'Daily_Hours', 'Weekly_Days', 'Work_Hours_Period', 'FTE', 'Pay_Scale', 
    'Pay_Scale_Text', 'Pay_Scale_Area', 'Pay_Scale_Area_Text', 'Grade', 'Point', 
    'Number', 'Amount', 'Currency', 'Staff_Id_y'
]

wage_short = wage_nodupe_relandfprel_df.drop(columns=columns_to_drop)

# Select only the required columns from got_bonus_df along with the column to join on
got_bonus_df_subset = got_bonus_df[['Personnel_No', 'Full_Total_Bonus', 'Total_Bonus_Count']]

# Perform the left merge
ws_merged_df = pd.merge(wage_short, got_bonus_df_subset, how='left', on='Personnel_No')

columns_to_drop = ['Full_Total_Bonus_x', 'Total_Bonus_Count_x']
ws_merged_df.drop(columns_to_drop, axis=1, errors='ignore', inplace=True)

# Rename columns only if they exist in the DataFrame
columns_to_rename = {'Full_Total_Bonus_y': 'Full_Total_Bonus', 'Total_Bonus_Count_y': 'Total_Bonus_Count'}
ws_merged_df.rename(columns=columns_to_rename, errors='ignore', inplace=True)


In [ ]:
# Define the directory and filename
directory = r'directory'
filename = 'filename'
# Create the full path to the CSV file
file_path = os.path.join(directory, filename)
ws_merged_df['Include Flag'] = ws_merged_df['Employee_Classification'].apply(lambda x: 'Yes' if x == 'Full-pay relevant' else '')
ws_merged_df.rename(columns={'Staff_Id_x': 'Staff_Id'}, inplace=True)
# Save the DataFrame to the CSV file
#ws_merged_df.to_csv(file_path, index=False)

In [ ]:
# Get descriptive statistics for 'HourlyExcSS'. Expected values mean and median 20 - 25 with a max of around 130
stats_hourly_exc_ss = wage_nodupe_df['HourlyExcSS'].describe()
stats_hourly_inc_ss = wage_nodupe_df['HourlyIncSS'].describe()

In [ ]:
wage_nodupe_df['HourlyIncSS'].describe()

In [ ]:
# spot check for the highest hourly rate - if this returns an unexpected row then further checks needed
idx_max = wage_nodupe_df['HourlyExcSS'].idxmax()
max_HourlyExcSS_row = wage_nodupe_df.loc[idx_max]
specified_columns_df = max_HourlyExcSS_row[['Title', 'Forename', 'Surname', 'HourlyExcSS']]

In [ ]:
# Group by 'Gender' and calculate median and mean of 'HourlyExcSS'
gender_stats = wage_nodupe_df.groupby('Gender')['HourlyExcSS'].agg(['median', 'mean']).reset_index()

# Calculate % difference of Female compared to Male for median and mean
female_median = gender_stats.loc[gender_stats['Gender'] == 'Female', 'median'].values[0]
male_median = gender_stats.loc[gender_stats['Gender'] == 'Male', 'median'].values[0]
female_mean = gender_stats.loc[gender_stats['Gender'] == 'Female', 'mean'].values[0]
male_mean = gender_stats.loc[gender_stats['Gender'] == 'Male', 'mean'].values[0]

# Use absolute value to express the difference as a positive number
percent_diff_median = abs((female_median - male_median) / male_median) * 100
percent_diff_mean = abs((female_mean - male_mean) / male_mean) * 100

# Add row for % difference
percent_diff_row = pd.DataFrame({
    'Gender': ['% difference'],
    'median': [percent_diff_median],
    'mean': [percent_diff_mean]
})

# Combine tables
result_table = pd.concat([gender_stats, percent_diff_row], ignore_index=True)
result_table

## All calculations are now done

In [ ]:
df_all = wage_nodupe_df

<div class="alert alert-block alert-info">Use ydata profiling for EDA now wage df is concatenated - uncomment to use</div>

In [ ]:
#profile = ProfileReport(wage_nodupe_df, title="Profiling Report")
#profile.to_notebook_iframe()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
# Create a boxplot of the female and male values for HourlyExcSS using seaborn with £ on x-axis
plt.figure(figsize=(16, 9))  # Adjusting figure size to 16:9 ratio
ax = sns.boxplot(x='HourlyExcSS', y='Gender', data=df_all, palette="Set3", orient='h')
plt.title('Boxplot of HourlyExcSS by Gender')
ax.set(xlabel='HourlyExcSS (£)', ylabel='Gender')  # Adding £ symbol to the x-axis label
# Format x-axis ticks with £ symbol
fmt = '£{x:,.0f}'
tick = plt.FuncFormatter(lambda x, _: fmt.format(x=x))
ax.xaxis.set_major_formatter(tick)
plt.show()

## FUNCTION TO CREATE TABLES SHOWING MEAN AND MEDIAN DIFFERENCES

In [ ]:
def calculate_stats_compared_to_group(df, characteristic, metric, comparator):
    # Group by characteristic and calculate median, mean, count for the metric
    stats = df.groupby(characteristic)[metric].agg(['median', 'mean', 'count']).reset_index()

    # Calculate the total count to use for percentage calculation
    total_count = stats['count'].sum()

    # Calculate % count for each group and add it as a new column to stats
    stats['% count'] = (stats['count'] / total_count) * 100

    # Calculate the values for the comparator group
    comparator_stats = stats[stats[characteristic] == comparator]

    # Initialize a DataFrame to hold the percent differences
    percent_diffs = pd.DataFrame(columns=[characteristic, 'median', 'mean', 'count', '% count'])

    for _, row in stats.iterrows():
        if row[characteristic] != comparator:
            # Calculate % difference for median and mean compared to the comparator (reverse the order of subtraction)
            percent_diff_median = ((comparator_stats['median'].values[0] - row['median']) / comparator_stats['median'].values[0]) * 100
            percent_diff_mean = ((comparator_stats['mean'].values[0] - row['mean']) / comparator_stats['mean'].values[0]) * 100
            
            # Calculate count difference and % count difference compared to the comparator
            diff_count = row['count'] - comparator_stats['count'].values[0]
            percent_diff_count = row['% count'] - comparator_stats['% count'].values[0]
        
            # Create a new row as a DataFrame to add to percent_diffs
            new_row = pd.DataFrame({
                characteristic: [f"difference from {comparator} ({row[characteristic]})"],
                'median': [percent_diff_median],
                'mean': [percent_diff_mean],
                'count': [diff_count],
                '% count': [percent_diff_count]
            })
        
            # Drop empty or all-NA columns from new_row before concatenating
            new_row = new_row.dropna(axis='columns', how='all')
        
            # Use pd.concat with the cleaned new_row
            percent_diffs = pd.concat([percent_diffs, new_row], ignore_index=True)
    
    # Combine tables
    result_table = pd.concat([stats, percent_diffs], ignore_index=True)

    # Format the result_table
    result_table['mean'] = result_table.apply(lambda x: f"£{x['mean']:.2f}" if not x[characteristic].startswith('difference from') else f"{x['mean']:.1f}%", axis=1)
    result_table['median'] = result_table.apply(lambda x: f"£{x['median']:.2f}" if not x[characteristic].startswith('difference from') else f"{x['median']:.1f}%", axis=1)
    result_table['count'] = result_table['count'].apply(lambda x: f"{x:.0f}")
    result_table['% count'] = result_table['% count'].apply(lambda x: f"{x:.1f}%")

    return result_table

## Mean and median salary sacrifice

In [ ]:
# Filter the DataFrame for rows where Total_Salary_Sacrifice > 0
salary_sacrifice_df = df_all[df_all['Total_Salary_Sacrifice'] > 0]

# Group by 'Gender' and calculate the required statistics
gender_stats = salary_sacrifice_df.groupby('Gender').agg(
    Count=('Total_Salary_Sacrifice', 'size'),
    Mean=('Total_Salary_Sacrifice', 'mean'),
    Median=('Total_Salary_Sacrifice', 'median')
)

# Calculate the percentage of each gender
gender_stats['Percentage'] = (gender_stats['Count'] / gender_stats['Count'].sum()) * 100

# Format the 'Percentage', 'Mean', and 'Median' columns
gender_stats['Percentage'] = gender_stats['Percentage'].map('{:.1f}%'.format)
gender_stats['Mean'] = gender_stats['Mean'].map('£{:.2f}'.format)
gender_stats['Median'] = gender_stats['Median'].map('£{:.2f}'.format)
salary_sacrifice_table = gender_stats

In [ ]:
all_eth_incss = calculate_stats_compared_to_group(df_all, 'EthnicityE5', 'HourlyIncSS', 'White')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
# Create a boxplot of the female and male values for HourlyExcSS using seaborn with £ on x-axis
plt.figure(figsize=(16, 9))  # Adjusting figure size to 16:9 ratio
ax = sns.boxplot(x='HourlyExcSS', y='EthnicityE5', data=wage_nodupe_df, palette="Set3", orient='h')
plt.title('Boxplot of HourlyExcSS by EthnicityE5')
ax.set(xlabel='HourlyExcSS (£)', ylabel='EthnicityE5')  # Adding £ symbol to the x-axis label
# Format x-axis ticks with £ symbol
fmt = '£{x:,.0f}'
tick = plt.FuncFormatter(lambda x, _: fmt.format(x=x))
ax.xaxis.set_major_formatter(tick)
plt.show()

In [ ]:
bonus_all_gen = calculate_stats_compared_to_group(got_bonus_df, 'Gender', 'Full_Total_Bonus', 'Male')

In [ ]:
bonus_stats = got_bonus_df['Full_Total_Bonus'].describe()

In [ ]:
df_nusl = df_all[df_all['Payroll_Area_SL'] == 'M2']

In [ ]:
all_gen_excss = calculate_stats_compared_to_group(df_all, 'Gender', 'HourlyExcSS', 'Male')
all_gen_incss = calculate_stats_compared_to_group(df_all, 'Gender', 'HourlyIncSS', 'Male')
nusl_gen_excss = calculate_stats_compared_to_group(df_nusl, 'Gender', 'HourlyExcSS', 'Male')

## Calculate Quartiles

In [ ]:
def calculate_and_add_quartiles(df, filter_condition=None):
    # Apply the filtering condition to the DataFrame if provided
    if filter_condition is not None:
        filtered_df = df[filter_condition]
    else:
        filtered_df = df.copy()  # Make a copy of the original DataFrame if no filter condition provided
    
    # Define quartile labels
    labels = ["Lower", "Lower Mid", "Upper Mid", "Upper"]
    
    # Calculate quartiles using the specified labels and method
    filtered_df['Quartile'] = pd.qcut(filtered_df['HourlyIncSS'].rank(method='first'), q=4, labels=labels)
    
    # Create a dictionary mapping Personnel_No to Quartile
    quartile_mapping = dict(filtered_df[['Personnel_No', 'Quartile']].values)
    
    # Add the 'Quartile' column to the original DataFrame using map
    df['Quartile'] = df['Personnel_No'].map(quartile_mapping)
    
    return df

In [ ]:
all_all_q = calculate_and_add_quartiles(df_all)
#all_ps_q = calculate_and_add_quartiles(df_all, (df_all['Employee_Type'] == 'Professional Support'))
# UNN ['Payroll_Area_SL'] == 'M1']
# NUSL ['Payroll_Area_SL'] == 'M2']
# ACA ['Employee_Type'] == 'Academic']
# PS ['Employee_Type'] == 'Professional Support']

## Summarise quartiles

In [ ]:
def quartile_summary(df, characteristic_column, quartile_column):
    # Define custom order for Quartile
    custom_order = ["Lower", "Lower Mid", "Upper Mid", "Upper"]
    
    # Group the DataFrame by the quartile and characteristic columns
    grouped = df.groupby([quartile_column, characteristic_column])['Personnel_No'].count().reset_index()
    
    # Rename the 'Personnel_No' column to 'Count'
    grouped.rename(columns={'Personnel_No': 'Count'}, inplace=True)
    
    # Calculate the total count of Personnel_No for each characteristic
    total_counts = grouped.groupby(characteristic_column)['Count'].sum()
    
    # Calculate the percentage of each quartile count relative to the total count for each characteristic
    grouped['Percentage'] = grouped.apply(lambda row: (row['Count'] / total_counts[row[characteristic_column]]) * 100, axis=1)
    
    # Format the 'Percentage' column to one decimal place
    grouped['Percentage'] = grouped['Percentage'].round(1)
    
    # Reorder the columns as Quartile, Characteristic, Count, Percentage
    grouped = grouped[['Quartile', characteristic_column, 'Count', 'Percentage']]
    
    # Set custom order for Quartile column
    grouped['Quartile'] = pd.Categorical(grouped['Quartile'], categories=custom_order, ordered=True)
    
    # Sort the DataFrame based on the custom order of Quartile
    grouped = grouped.sort_values(by='Quartile')
    
    return grouped

In [ ]:
gender_quartiles = quartile_summary(all_all_q, 'Gender', 'Quartile')

## Export pay gap tables to excel

In [ ]:
from openpyxl import Workbook
def create_excel_with_tables(dfs, sheet_names, file_path, fill_missing_values=False, fill_value=None, max_string_length=None):
    # Create a new Excel workbook
    wb = Workbook()
    wb.remove(wb.active)  # Remove the default sheet

    for df, sheet_name in zip(dfs, sheet_names):
        # Ensure that the df is a DataFrame or a Series
        if not isinstance(df, (pd.DataFrame, pd.Series)):
            print(f"Skipping {sheet_name}: Not a DataFrame or Series")
            continue

        if isinstance(df, pd.Series):
            # Convert Series to DataFrame
            df = df.to_frame()

        # Fill missing values if requested
        if fill_missing_values:
            fill_value_to_use = fill_value if fill_value is not None else 0
            for column in df.select_dtypes(include=['category']).columns:
                if fill_value_to_use not in df[column].cat.categories:
                    df[column] = df[column].cat.add_categories([fill_value_to_use])
            df.fillna(fill_value_to_use, inplace=True)

        # Reset the index so that the index is included as a column
        df = df.reset_index()

        # Create a new sheet
        ws = wb.create_sheet(title=sheet_name)

        # Write headers
        for col, header in enumerate(df.columns, 1):
            ws.cell(row=1, column=col, value=header)

        # Write data rows
        for row, data_row in enumerate(df.itertuples(index=False), 2):
            for col, value in enumerate(data_row, 1):
                if max_string_length and isinstance(value, str):
                    value = value[:max_string_length]
                ws.cell(row=row, column=col, value=value)

    # Save the workbook to the specified file path
    wb.save(file_path)

# Example usage with filling missing values and max_string_length
dfs = [
    wage_short,
    stats_hourly_exc_ss,
    stats_hourly_inc_ss,
    salary_sacrifice_table,
    bonus_all_gen,
    bonus_stats,
    all_gen_excss,
    all_gen_incss,
    gender_quartiles
]
sheet_names = ["Main", "Casual_Pay","Casual_Hours", "Salary Sacrifice","Bonus Gender",
               "Bonus Stats","Gen Gap Ex","Gen Gap Inc",
               "Quartiles"]
# create_excel_with_tables(dfs, sheet_names, r"filepath here", fill_missing_values=True, fill_value=None, max_string_length=255)

print("All processing complete.")